In [3]:
from src.features.videos.objects import object as object_helper
from src.models.lda import lda as lda_helper
from src import constants

import pandas as pd
import numpy as np

import psycopg2

%load_ext autoreload
%autoreload 2

/Users/claasmeiners/.virtualenvs/video_article_retrieval/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
object_occurence_rate = object_helper.get_occurence_sum()
object_occurence_rate.head()

,platform,id,person,bicycle,car,motorbike,aeroplane,bus,train,truck,...,toaster,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush
0,facebook,100001732932719/1647247215343027,10.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,facebook,100002259829821/1786627524755891,166.0,0.0,1951.0,0.0,1.0,1.0,0.0,260.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,facebook,100002259829821/1786745844744059,152.0,0.0,74.0,0.0,0.0,0.0,0.0,37.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,facebook,100004527049328/972499912910901,298.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,facebook,100004527049328/973448889482670,198.0,0.0,426.0,0.0,0.0,1.0,0.0,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
object_occurence_rate = object_helper.divide_by_duration(object_occurence_rate)
object_occurence_rate.head()

,platform,id,person,bicycle,car,motorbike,aeroplane,bus,train,truck,...,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush,duration
0,facebook,100001732932719/1647247215343027,0.000339,0.0,0.000000,0.000034,0.000000,0.000034,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29510
1,facebook,100002259829821/1786627524755891,0.000203,0.0,0.002390,0.000000,0.000001,0.000001,0.0,0.000319,...,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,816264
2,facebook,100002259829821/1786745844744059,0.002719,0.0,0.001324,0.000000,0.000000,0.000000,0.0,0.000662,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55911
3,facebook,100004527049328/972499912910901,0.001683,0.0,0.000367,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177064
4,facebook,100004527049328/973448889482670,0.001430,0.0,0.003077,0.000000,0.000000,0.000007,0.0,0.001632,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138453


In [7]:
features_connection = psycopg2.connect(database="video_article_retrieval", user="postgres")
average_topics = pd.read_sql("SELECT * FROM average_topics", con=features_connection)
average_topics.head()

,id,platform,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,fox26houston/10157181811750348,facebook,0.0,0.767010,0.0,0.153838,0.046777,0.000000,0.005562,0.020166,0.000000,0.00393
1,jessica.salazar.338/2111423425558048,facebook,0.0,0.099226,0.0,0.158874,0.159987,0.016518,0.000000,0.020300,0.543781,0.00000
2,JoyNewsOnTV/2167529583318936,facebook,0.0,0.000000,0.0,0.000000,0.304423,0.000000,0.675539,0.000000,0.017435,0.00000
3,100009629315589/737936519870652,facebook,0.0,0.546476,0.0,0.000000,0.138599,0.000000,0.000000,0.201420,0.106360,0.00000
4,185998671553/10157087493556554,facebook,0.0,0.000000,0.0,0.264172,0.359579,0.117422,0.000000,0.255583,0.000000,0.00000


In [8]:
# Right join on the y dataframe to only keep the videos where we have detected the objects
data = pd.merge(average_topics, object_occurence_rate,  how='right', left_on=['id','platform'], right_on = ['id','platform'])
data.head()

,id,platform,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush,duration
0,fox26houston/10157181811750348,facebook,0.0,0.767010,0.0,0.153838,0.046777,0.000000,0.005562,0.020166,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,449536
1,jessica.salazar.338/2111423425558048,facebook,0.0,0.099226,0.0,0.158874,0.159987,0.016518,0.000000,0.020300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82047
2,JoyNewsOnTV/2167529583318936,facebook,0.0,0.000000,0.0,0.000000,0.304423,0.000000,0.675539,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196371
3,100009629315589/737936519870652,facebook,0.0,0.546476,0.0,0.000000,0.138599,0.000000,0.000000,0.201420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,218823
4,185998671553/10157087493556554,facebook,0.0,0.000000,0.0,0.264172,0.359579,0.117422,0.000000,0.255583,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38380


In [9]:
topics = lda_helper.get_topic_columns(10)
objects = [label.replace(" ", "_") for label in constants.COCO_CLASS_NAMES]
data[topics].head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,0.0,0.767010,0.0,0.153838,0.046777,0.000000,0.005562,0.020166,0.000000,0.00393
1,0.0,0.099226,0.0,0.158874,0.159987,0.016518,0.000000,0.020300,0.543781,0.00000
2,0.0,0.000000,0.0,0.000000,0.304423,0.000000,0.675539,0.000000,0.017435,0.00000
3,0.0,0.546476,0.0,0.000000,0.138599,0.000000,0.000000,0.201420,0.106360,0.00000
4,0.0,0.000000,0.0,0.264172,0.359579,0.117422,0.000000,0.255583,0.000000,0.00000


In [10]:
data[objects].head()

,person,bicycle,car,motorbike,aeroplane,bus,train,truck,boat,traffic_light,...,toaster,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush
0,0.001671,0.000000,0.006709,0.000004,0.0,0.00008,0.0,0.00117,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.002974,0.000024,0.000000,0.000024,0.0,0.00000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.002353,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000932,0.000000,0.000119,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.001980,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# models
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


X_train, X_test, y_train, y_test = train_test_split(data[objects], data[topics], test_size=0.2)
models = [
    DummyRegressor(strategy='mean'),
    LinearRegression(),
    RandomForestRegressor(),
    KNeighborsRegressor()
]

for model in models:
    model.fit(data[objects], data[topics])
    scores = cross_val_score(model, data[objects], data[topics], cv=5, scoring='neg_mean_squared_error')
    # print('Coefficients: \n', regr.coef_)
    # print(model.score(X_test, y_test))
    # y_pred = regr.predict(X_test)
    # print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    # print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

Accuracy: -0.023 (+/- 0.001)
Accuracy: -0.025 (+/- 0.008)


/Users/claasmeiners/.virtualenvs/video_article_retrieval/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: -0.022 (+/- 0.002)
Accuracy: -0.024 (+/- 0.001)


In [100]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 0.26
Variance score: -11.31


In [ ]:
models = [linear_model.LinearRegression()]
for model in models:
    model.fit(data[objects], data[topics])
    scores = cross_val_score(model, data[objects], data[topics], cv=5)
    # print('Coefficients: \n', regr.coef_)
    # print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))
    # print(model.score(X_test, y_test))
    print(scores)